In [ ]:
import json 
import uuid
from uuid import uuid4
# data manipulation
import pandas as pd 
import string
# normalize nested JSON files
from pandas.io.json import json_normalize
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import PyPDF2, urllib.request , nltk , textract
from io import BytesIO
import json
#import weasyprint
import pandas as pd
from pandas.io.json import json_normalize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
from fuzzywuzzy import fuzz
import json
import logging
import multiprocessing as mp
import numpy as np
import pandas as pd
import os
import re
import regex
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')  
nltk.download('punkt')

In [ ]:
json_file_path = "OKU-fact-sheets-out.json"
with open(json_file_path, 'r') as json_file: 
    data = json.load(json_file)

In [ ]:
#grab the contents from OKU json
links = [x['link'] for x in data]
texts = [x['content'] for x in data]
displaydate = [x['displaydate'] for x in data]

In [ ]:
#make the data into dataframe
df1 = pd.DataFrame(links)
df1.columns = ['extension_links']
df2 = pd.DataFrame(texts)
df_new=df2.copy()
df_new['text'] = df_new[df_new.columns[0:110]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)
df_new = df_new.drop(df_new.columns[0:111],axis=1)
df3 = pd.DataFrame(displaydate)
df3.columns = ['Displaydate']
all_data = pd.concat([df1,df_new,df3],axis=1, join='inner')
all_data['row_num'] = np.arange(len(all_data))
all_data

In [ ]:
#create unique id based on date,OKU, and the unique number of document
all_data['unique_id'] = all_data['Displaydate'].astype(str)+'-'+all_data['row_num'].astype(str)+'-'+'OKU'
all_data

In [ ]:
#save it to csv format
all_data.to_csv("all_data.csv")


In [ ]:
#make each row to separate json file
df = pd.read_csv('all_data.csv', header=0, dtype=str) 
df['unique_id'] = df['unique_id'].map(str)
df.set_index('unique_id', inplace=True)
for unique_id, data in df.iterrows():
    data.to_csv(rf'C:\Users\Xiangyu Ren\Documents\NASS project\{unique_id}.json')
